In [1]:
!nvidia-smi

Sat Feb 22 17:45:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 6000                On  |   00000000:AF:00.0 Off |                  Off |
| 33%   29C    P8              3W /  260W |    7264MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os

HOME = os.getcwd()
print("HOME:", HOME)

HOME: /storage/scratch1/2/dkhattak6


In [4]:
!echo $PYSPARK_PYTHON

In [5]:
import sys

print(sys.version)

3.12.5 (main, Dec 19 2024, 14:46:22) [GCC 11.4.1 20231218 (Red Hat 11.4.1-3)]


In [6]:
import os
import sys

!echo $PYSPARK_PYTHON $PYSPARK_DRIVER_PYTHON
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [7]:
!echo $PYSPARK_PYTHON $PYSPARK_DRIVER_PYTHON

/storage/scratch1/2/dkhattak6/.venv/bin/python3 /storage/scratch1/2/dkhattak6/.venv/bin/python3


In [8]:
import torch
import torch._C

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from torch.utils.cpp_extension import CUDA_HOME

print(CUDA_HOME)

/usr/local/cuda


In [9]:
print(torch._C)

<module 'torch._C' from '/storage/scratch1/2/dkhattak6/.venv/lib/python3.12/site-packages/torch/_C.cpython-312-x86_64-linux-gnu.so'>


In [10]:
import os

SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))

/storage/scratch1/2/dkhattak6/weights/sam_vit_h_4b8939.pth ; exist: True


In [11]:
import os

GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(
    HOME, "weights", "groundingdino_swint_ogc.pth"
)
print(
    GROUNDING_DINO_CHECKPOINT_PATH,
    "; exist:",
    os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH),
)

/storage/scratch1/2/dkhattak6/weights/groundingdino_swint_ogc.pth ; exist: True


In [ ]:
GROUNDING_DINO_CONFIG_PATH = os.path.join(
    HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
)
print(
    GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH)
)

/storage/scratch1/2/dkhattak6/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, BinaryType, StringType, MapType

In [14]:
print(torch.version.cuda)

11.8


In [15]:
spark = SparkSession.builder.appName("Print PySpark Version").getOrCreate()
print("PySpark Version:", spark.version)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/22 17:46:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version: 3.5.4


In [16]:
print(DEVICE)

cuda


In [17]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import Model

grounding_dino_model = Model(
    model_config_path=GROUNDING_DINO_CONFIG_PATH,
    model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH,
)

/storage/scratch1/2/dkhattak6/GroundingDINO


/storage/scratch1/2/dkhattak6/.venv/lib/python3.12/site-packages/timm-1.0.14-py3.12.egg/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/storage/scratch1/2/dkhattak6/.venv/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


In [ ]:
from segment_anything import sam_model_registry, SamPredictor

SAM_ENCODER_VERSION = "vit_h"
sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to(
    device=DEVICE
)
sam_predictor = SamPredictor(sam)

In [19]:
CLASSES = [
    "leaf",
    "flower",
    "sand",
    "wood",
    "stone",
    "tape",
    "plant",
    "tree",
    "rock",
    "vegetation",
]
BOX_TRESHOLD = 0.15
TEXT_TRESHOLD = 0.1
include_class_ids = {0, 1, 6, 7, 9}  # only include positive classes

In [20]:
from typing import List


def enhance_class_name(class_names: List[str]) -> List[str]:
    return [f"all {class_name}s" for class_name in class_names]

In [23]:
import numpy as np
from segment_anything import SamPredictor
import cv2
import supervision as sv


def segment(
    sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray
) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(box=box, multimask_output=True)
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

In [25]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession


def image_to_bytes(image_array, ext=".png"):
    """
    Encode an image or mask array to bytes using OpenCV.
    Use PNG for masks (lossless) and JPG for the original image.
    """
    success, buffer = cv2.imencode(ext, image_array)
    if not success:
        raise ValueError("Image encoding failed!")
    return buffer.tobytes()

In [ ]:
from tqdm.notebook import tqdm
from pathlib import Path

counter = 0
IMAGES_DIRECTORY = os.path.join(HOME, "data")
HOMEPATH = Path(os.path.expanduser("~/shared/plantclef/data"))
IMAGES_DIRECTORY = os.path.join(HOMEPATH, "test_2024/images")
IMAGES_EXTENSIONS = ["jpg"]
MASK_IMAGE_PATH = f"{HOME}/masks_2024/"
records = []
images = {}
annotations = {}
image_paths = sv.list_files_with_extensions(
    directory=IMAGES_DIRECTORY, extensions=IMAGES_EXTENSIONS
)

for image_path in tqdm(image_paths):
    image_name = image_path.name
    image_path = str(image_path)
    image = cv2.imread(image_path)

    detections = grounding_dino_model.predict_with_classes(
        image=image,
        classes=enhance_class_name(class_names=CLASSES),
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD,
    )
    detections = detections[detections.class_id is not None]
    detections.mask = segment(
        sam_predictor=sam_predictor,
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
        xyxy=detections.xyxy,
    )
    images[image_name] = image
    annotations[image_name] = detections
    titles = [CLASSES[class_id] for class_id in detections.class_id]
    merged_masks = []
    merged_titles = []
    per_class_mask_dict = {}

    grouped = {}  # {class_id: [mask1, mask2, ...]}
    for mask, class_id in zip(detections.mask, detections.class_id):
        if class_id not in grouped:
            grouped[class_id] = []
        grouped[class_id].append(mask)

    for class_id, masks in grouped.items():
        if class_id in include_class_ids:
            if len(masks) > 0:
                merged_mask = np.any(
                    np.stack(masks, axis=0), axis=0
                )  # Basically doing a Logical OR over the masks
                merged_masks.append(merged_mask)
                merged_titles.append(CLASSES[class_id])
                per_class_mask_dict[CLASSES[class_id]] = (
                    merged_mask.astype(np.uint8)
                ) * 255

    if len(merged_masks) > 0:
        final_mask = np.any(np.stack(merged_masks, axis=0), axis=0)
        final_mask_uint8 = (final_mask.astype(np.uint8)) * 255
        segmented_image = cv2.bitwise_and(image, image, mask=final_mask_uint8)
    else:
        segmented_image = image
        # print("No masks found for image " + str(image_name))
        final_mask_uint8 = None
        counter += 1
        per_class_mask_dict = {}

    cv2.imwrite(MASK_IMAGE_PATH + str(image_name), segmented_image)
    original_bytes = image_to_bytes(image, ext=".jpg")
    final_mask_bytes = (
        image_to_bytes(final_mask_uint8, ext=".png")
        if final_mask_uint8 is not None
        else None
    )

    class_masks_bytes = {}
    for cls, mask in per_class_mask_dict.items():
        class_masks_bytes[cls] = image_to_bytes(mask, ext=".png")

    # Create a record for this image
    record = (image_name, original_bytes, final_mask_bytes, class_masks_bytes)
    records.append(record)

print("Could not create masks for " + str(counter) + " images")

  0%|          | 0/1695 [00:00<?, ?it/s]

In [34]:
print(MASK_IMAGE_PATH)

/storage/home/hcoda1/2/dkhattak6/shared/plantclef/data/masks_test_2024/


In [28]:
schema = StructType(
    [
        StructField("image_name", StringType(), False),
        StructField("image", BinaryType(), False),
        StructField("final_mask", BinaryType(), True),
        StructField("class_masks", MapType(StringType(), BinaryType()), True),
    ]
)

In [ ]:
spark = SparkSession.builder.appName("SegmentedImageStorage").getOrCreate()


schema = StructType(
    [
        StructField("image_name", StringType(), False),
        StructField("image", BinaryType(), False),
        StructField("final_mask", BinaryType(), True),
        StructField("class_masks", MapType(StringType(), BinaryType()), True),
    ]
)

# Create the DataFrame from the list of records
df = spark.createDataFrame(records, schema=schema)
image_final_df = df.select(
    F.col("image_name"), F.col("image"), F.col("final_mask"), F.col("class_masks")
).repartition(500)
image_final_df.write.mode("overwrite").parquet("segmented_images.parquet")

In [37]:
from pathlib import Path

root = Path(os.path.expanduser("~"))
data_path = f"{root}/shared/plantclef/data/parquet/"

# Define the path to the train and test parquet files

test_path = f"{data_path}/test"

# Read the parquet files into a spark DataFrame

test_df = spark.read.parquet(test_path)

# Show the data
test_df.printSchema()

test_df.show(n=5)

root
 |-- image_name: string (nullable = true)
 |-- path: string (nullable = true)
 |-- data: binary (nullable = true)



+--------------------+--------------------+--------------------+
|          image_name|                path|                data|
+--------------------+--------------------+--------------------+
|CBN-Pla-B3-201907...|/test/data/PlantC...|[FF D8 FF E0 00 1...|
|CBN-PdlC-E5-20180...|/test/data/PlantC...|[FF D8 FF E0 00 1...|
|CBN-PdlC-B1-20140...|/test/data/PlantC...|[FF D8 FF E0 00 1...|
|CBN-Pla-D4-201507...|/test/data/PlantC...|[FF D8 FF E0 00 1...|
|CBN-PdlC-F3-20190...|/test/data/PlantC...|[FF D8 FF E0 00 1...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [49]:
# import argparse
# def parse_args():
#     """Parse command line arguments."""
#     home_dir = HOME

#     parser = argparse.ArgumentParser(
#         description="Process test image dataset stored on PACE."
#     )
#     parser.add_argument(
#         "--cores",
#         type=int,
#         default=os.cpu_count(),
#         help="Number of cores used in Spark driver",
#     )
#     parser.add_argument(
#         "--memory",
#         type=str,
#         default="16g",
#         help="Amount of memory to use in Spark driver",
#     )
#     return parser.parse_args()

In [29]:
# args = parse_args()
import time
from pathlib import Path

# Initialize Spark
# Replace get_spark with your appropriate Spark initialization function if needed.
cores = (os.cpu_count(),)
memory = (os.environ.get("PYSPARK_DRIVER_MEMORY", "16g"),)
executor_memory = (os.environ.get("PYSPARK_EXECUTOR_MEMORY", "1g"),)
local_dir = (os.environ.get("SPARK_LOCAL_DIR", "/tmp"),)
app_name = "clef"
local_dir = f"{local_dir}/{int(time.time())}"
Path(local_dir).mkdir(parents=True, exist_ok=True)
builder = (
    SparkSession.builder.config("spark.driver.memory", memory)
    .config("spark.executor.memory", executor_memory)
    .config("spark.driver.cores", cores)
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.driver.maxResultSize", "8g")
    .config("spark.local.dir", local_dir)
)
builder.appName("SegmentedImageStorage").getOrCreate()
df = spark.createDataFrame(records, schema=schema)
image_final_df = df.select(
    F.col("image_name"), F.col("image"), F.col("final_mask"), F.col("class_masks")
).repartition(500)
image_final_df.write.mode("overwrite").parquet("segmented_images.parquet")

25/02/22 17:52:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/02/22 17:52:43 WARN TaskSetManager: Stage 0 contains a task of very large size (8230 KiB). The maximum recommended task size is 1000 KiB.


In [32]:
HOMEPATH = Path(os.path.expanduser("~/shared/plantclef"))
print(HOMEPATH)

/storage/home/hcoda1/2/dkhattak6/shared/plantclef
